# split data

In [1]:
import torch
import numpy as np
import json
import os
import cv2
import random
import glob
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd

import tqdm
import json
import pickle
from shutil import copy

In [2]:
!mkdir /root/dataset
!mkdir /root/dataset/fold1
!mkdir /root/dataset/fold1/images
!mkdir /root/dataset/fold1/images/train
!mkdir /root/dataset/fold1/images/val_all
!cp -r /root/dataset/fold1/images /root/dataset/fold1/labels

!cp -r /root/dataset/fold1 /root/dataset/fold2 
!cp -r /root/dataset/fold1 /root/dataset/fold3 
!cp -r /root/dataset/fold1 /root/dataset/fold4 
!cp -r /root/dataset/fold1 /root/dataset/fold5

# !mkdir logs


In [3]:
image_folder = '/root/train/'   # thư mục image gốc 
csv_train_only_box = '/root/train_only_box.csv'
data_p_dir = '/root/data.p'
train_all = True
fold = 0 # fold ở đây là từ 0 -> 4  nhưng ở folder là từ 1 -> 5

## TRAIN

In [4]:
df = pd.read_csv(csv_train_only_box)
with open(data_p_dir, 'rb') as fp:
    data_ = pickle.load(fp)    

In [5]:
data = {}

for i in range(len(df)):
    image_id = df.iloc[i]['image_id']
    if image_id not in data:
        data[image_id] = {}
    rad_id = df.iloc[i]['rad_id']
    
    if rad_id not in data[image_id]:
        data[image_id][rad_id] = []
        
    if 'fold' not in data[image_id]:
        data[image_id]['fold'] = df.iloc[i]['fold']

    class_id = df.iloc[i]['class_id']
    if class_id != 14:
        width = data_['{}.dicom.png'.format(image_id)]['width']
        height = data_['{}.dicom.png'.format(image_id)]['height']
        xmax = df.iloc[i]['x_max']
        ymax = df.iloc[i]['y_max']
        xmin = df.iloc[i]['x_min']
        ymin = df.iloc[i]['y_min']

        x_center = ((xmax + xmin)//2) / width
        b_width = (xmax - xmin) / width
        
        y_center = ((ymax + ymin)//2) / height
        b_height = (ymax - ymin) / height
        if b_height == 0:
            print(ymax, ymin, height, image_id)
        
        data[image_id][rad_id].append([class_id, x_center, y_center, b_width, b_height])

In [6]:
for image_id in data:
    if data[image_id]['fold'] != fold or train_all:
        image_path = '/root/dataset/fold{}/images/train'.format(fold + 1)
        label_path = '/root/dataset/fold{}/labels/train'.format(fold + 1)
    else:
        image_path = '/root/dataset/fold{}/images/val_all'.format(fold+1)
        label_path = '/root/dataset/fold{}/labels/val_all'.format(fold+1)

    for rad_id in data[image_id]:
        if rad_id == 'fold':continue

        f= open(os.path.join(label_path, '{}_{}.txt'.format(image_id, rad_id)),"w+")

        for anno in data[image_id][rad_id]:
            f.write("{} {} {} {} {}\n".format(anno[0], anno[1], anno[2], anno[3], anno[4]))

        f.close()
        copy('{}/{}.jpg'.format(image_folder, image_id), '{}/{}_{}.jpg'.format(image_path, image_id, rad_id))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-ea8da633db69>", line 18, in <module>
    copy('{}/{}.png'.format(image_folder, image_id), '{}/{}_{}.png'.format(image_path, image_id, rad_id))
  File "/usr/lib/python3.7/shutil.py", line 248, in copy
    copyfile(src, dst, follow_symlinks=follow_symlinks)
  File "/usr/lib/python3.7/shutil.py", line 122, in copyfile
    copyfileobj(fsrc, fdst)
  File "/usr/lib/python3.7/shutil.py", line 79, in copyfileobj
    buf = fsrc.read(length)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceb

KeyboardInterrupt: 

# train

In [ ]:
nohup python3 -u train.py \
--img 1920 \
--batch 8 \
--epochs 50 \
--data vin_fold1.yaml \   # fix lại có này cho mỗi fold
--cfg yolov5x6_vin.yaml \
--hyp data/hyp.vin2.yaml \
--worker 16 \
--weight yolov5x6.pt \
--multi-scale > logs/log_fold1.log &  # nhớ thay đổi đưuòng dẫn log cho từng fold 

In [43]:
!python3 -u train.py \
--img 960 \
--batch 8 \
--epochs 50 \
--data vin_fold1.yaml \   
--cfg yolov5x6_vin.yaml \
--hyp data/hyp.vin2.yaml \
--worker 16 \
--weight yolov5x6.pt \
--multi-scale

SyntaxError: invalid syntax (<ipython-input-43-6931c4a2c638>, line 2)